In [1]:
# -*-  coding:utf-8 -*-


import time
import random
import urllib.request

def gethtml(url):
    
    my_headers = [
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
    "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10"]
    
    headers ={'User-Agent': random.choice(my_headers),'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
    
    try:        
        req = urllib.request.Request(url, headers=headers)     
        result=urllib.request.urlopen(req).read().decode("utf-8")
        
    except Exception as e:
        print(e)
   
    return(result)

In [2]:
urlfirst = 'https://p.comments.youku.com/ycp/comment/pc/commentList?jsoncallback=n_commentList&app=100-DDwODVkv&objectId=1061201689&objectType=1&listType=0&currentPage=1&pageSize=30&sign=7c41cc8cd6e779094041e2d770d0204d&time=1564760892'


In [3]:
#找到每集的totalPage是多少
import re
import csv
resultfirst = gethtml(urlfirst)
totalPage1 = re.findall('totalPage":(.*?),"',str(resultfirst),re.S)#表格格式
print(totalPage1)


['85']


In [4]:
#构建urls
firsturls = ['https://p.comments.youku.com/ycp/comment/pc/commentList?jsoncallback=n_commentList&app=100-DDwODVkv&objectId=1061201689&objectType=1&listType=0&currentPage={}&pageSize=30&sign=7c41cc8cd6e779094041e2d770d0204d&time=1564760892'.format(str(i)) for i in range(1,int(totalPage1[0])+1)]


In [ ]:
import pandas as pd

for url in firsturls:
    time.sleep(2+float(random.randint(1, 50)) / 20)
    print(url)
    resultall = gethtml(url).encode('UTF-8').decode('unicode-escape','ignore')#unicode-escape
    
    def saveresult(resultall):
        AllComment = pd.DataFrame()
    
    #从第2行写入数据
        userId = re.findall('userId":(.*?),"userCode',str(resultall),re.S)
        userLevel = re.findall('userLevel":(.*?),"',str(resultall),re.S)
        userName = re.findall('userName":"(.*?)",',str(resultall),re.S)
        content = re.findall('content":"(.*?)",',str(resultall),re.S)
        createTime = re.findall('createTime":(.*?),"',str(resultall),re.S)
        vipInfo =re.findall('"vipInfo":{(.*?)}',str(resultall),re.S)
    
        AllComment['userId']=userId
        AllComment['userLevel']=userLevel
        AllComment['userName']=userName
        AllComment['content']=content
        AllComment['createTime']=createTime
        AllComment['vipInfo']=vipInfo

        
        return(AllComment)
    
    endresult = saveresult(resultall)
    
    alllist = endresult.values.tolist()#转成list，以list存入csv
    
    
    with open('ykcontents12.csv','a',newline='',encoding='gb18030') as csvfile:
        writer  = csv.writer(csvfile)
        for row in alllist:
            try:
                writer.writerow(row)
            except UnicodeEncodeError:
                pass
            continue
    
    '''用pandas的存入csv一直出现编码错误
endresult.to_csv('ykcresult11.csv',index=False, header=None, mode='a',encoding='utf_8_sig')#,encoding="gb18030"

'''
  